In [18]:
import pandas as pd
import tensorflow
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# for jupyter
%matplotlib inline

In [19]:
dt = pd.read_csv('healthcare-dataset-stroke-data.csv')
print(len(dt))
dt.head()

5110


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [20]:
dt.shape

(5110, 12)

In [21]:
47/249

0.18875502008032127

In [22]:

#d = pd.DataFrame()

In [23]:
# vamos remover os registros menores que 18 anos!
#dt = dt[dt['age'] >= 18]
d = dt.copy()

sexo = ['Male', 'Female','Other']

sim_nao = ['yes','no']
residencia = ['Urban', 'Rural']
smokes = ['formerly smoked', 'never smoked', 'smokes', 'Unknown']

d['gender'] = dt['gender'].apply( lambda x: sexo.index(x))
d['ever_married'] = dt['ever_married'].apply( lambda x: sim_nao.index(x.lower()))

d = d[ d['gender'] < 2 ]

lista = ['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked']
for i in lista:
    d['work_type_'+i] = dt['work_type'].apply(lambda x: 1 if x == i else 0)
d = d.drop('work_type',axis=1)
d['Residence_type'] = dt['Residence_type'].apply( lambda x: residencia.index(x))


for i in smokes:
    d['smoking_status_'+i] = dt['smoking_status'].apply(lambda x: 1 if x == i else 0)
d = d.drop('smoking_status',axis=1)

#

media_imc = d['bmi'].mean()
d['bmi'] = d['bmi'].apply(lambda x: media_imc if (str(x) == 'nan') else x)

d['age'] = d['age'].apply(lambda x: x/100)
#d['Senior'] = dt['age'].apply( lambda x: 1 if x >=60 else 0)
#d = d.drop('age',axis=1)

d['avg_glucose_level'] = d['avg_glucose_level'].apply(lambda x: (x-55)/(271.64 - 55) )

#d = d[ d['bmi'] < 60]

d['bmi'] = d['bmi']/100

print(d.shape[0])
d

5109


,id,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Private,work_type_Self-employed,work_type_Govt_job,work_type_children,work_type_Never_worked,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,smoking_status_Unknown
0,9046,0,0.67,0,1,0,0,0.801745,0.366000,1,1,0,0,0,0,1,0,0,0
1,51676,1,0.61,0,0,0,1,0.679514,0.288946,1,0,1,0,0,0,0,1,0,0
2,31112,0,0.80,0,1,0,1,0.235044,0.325000,1,1,0,0,0,0,0,1,0,0
3,60182,1,0.49,0,0,0,0,0.536512,0.344000,1,1,0,0,0,0,0,0,1,0
4,1665,1,0.79,1,0,0,1,0.549852,0.240000,1,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,1,0.80,1,0,0,0,0.132709,0.288946,0,1,0,0,0,0,0,1,0,0
5106,44873,1,0.81,0,0,0,0,0.324040,0.400000,0,0,1,0,0,0,0,1,0,0
5107,19723,1,0.35,0,0,0,1,0.129201,0.306000,0,0,1,0,0,0,0,1,0,0
5108,37544,0,0.51,0,0,0,1,0.513709,0.256000,0,1,0,0,0,0,1,0,0,0


In [24]:
"""
    E se nós removessemos uns registros? quais seriam os melhores registros?
    vamos tentar aleatoriamente!

dd = d.copy()
F = dd[ dd['stroke'] == 0]
# vamos criar uma coluna para remover

def PrintStatus():
    negativos = dd[dd['stroke']==0].shape[0]
    positivos = dd[dd['stroke']==1].shape[0]
    print("N:",negativos,"    P:",positivos,"  ->  ",(positivos*100)/(negativos+positivos))
PrintStatus()
remover = set()
while (len(remover) < 3000) and (len(remover) < len(F)):
    remover.add(F.sample().index[0])

dd = dd.drop(list(remover))
PrintStatus()

d = dd
"""

'\n    E se nós removessemos uns registros? quais seriam os melhores registros?\n    vamos tentar aleatoriamente!\n\ndd = d.copy()\nF = dd[ dd[\'stroke\'] == 0]\n# vamos criar uma coluna para remover\n\ndef PrintStatus():\n    negativos = dd[dd[\'stroke\']==0].shape[0]\n    positivos = dd[dd[\'stroke\']==1].shape[0]\n    print("N:",negativos,"    P:",positivos,"  ->  ",(positivos*100)/(negativos+positivos))\nPrintStatus()\nremover = set()\nwhile (len(remover) < 3000) and (len(remover) < len(F)):\n    remover.add(F.sample().index[0])\n\ndd = dd.drop(list(remover))\nPrintStatus()\n\nd = dd\n'

In [25]:
dr = d['stroke']
d =d.drop('stroke',axis=1)
d =d.drop('id',axis=1)


x_tmp,x_valida,y_tmp,y_valida = train_test_split(d, dr,test_size= 0.1, random_state=0)


from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(x_tmp, y_tmp)

from sklearn.model_selection import train_test_split
x_treino,x_teste,y_treino,y_teste = train_test_split(X_over,y_over,test_size=0.3,stratify=y_over,random_state=42)


In [33]:
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=0.001)

optimizer_SGD = tf.keras.optimizers.SGD(
    learning_rate=0.001,
    momentum=0.01,
    nesterov=False,
    name='SGD')

optimizer_Adagrad = tf.keras.optimizers.Adagrad(
    learning_rate=0.002,
    initial_accumulator_value=0.01,
    epsilon=1e-02,
    name="Adagrad",

)

In [58]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(10,activation=tf.nn.relu,input_shape=(x_treino.shape[1],)))
model.add(tf.keras.layers.Dense(20,activation=tf.nn.relu ))
model.add(tf.keras.layers.Dense(1,activation=tf.nn.sigmoid ))
model.summary()
model.compile(optimizer=optimizer_SGD,
              loss='MeanSquaredError',
              metrics=['accuracy','Recall'])

tf.summary.trace_on(graph=True, profiler=False)

model.fit(x_treino,y_treino,batch_size=180, epochs=8000,verbose=2,
          validation_split=0.8,
          )

teste  = model.evaluate(x_teste,  y_teste, verbose=2)
treino = model.evaluate(x_treino,  y_treino, verbose=2)



print()
print()

print("F1-score:", )

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 10)                180       
                                                                 
 dense_37 (Dense)            (None, 20)                220       
                                                                 
 dense_38 (Dense)            (None, 1)                 21        
                                                                 
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________
Epoch 1/8000
7/7 - 1s - loss: 0.2531 - accuracy: 0.5200 - recall: 0.9655 - val_loss: 0.2527 - val_accuracy: 0.5172 - val_recall: 0.9626 - 756ms/epoch - 108ms/step
Epoch 2/8000
7/7 - 0s - loss: 0.2531 - accuracy: 0.5200 - recall: 0.9655 - val_loss: 0.2526 - val_accuracy: 0.5174 - val_recall: 0.9626 - 51ms/epoch - 7ms/s

In [60]:
print(model.evaluate(x_valida,  y_valida, verbose=2))

16/16 - 0s - loss: 0.1803 - accuracy: 0.7143 - recall: 0.7333 - 28ms/epoch - 2ms/step
[0.18034029006958008, 0.7142857313156128, 0.7333333492279053]


In [37]:
"""
x_treino2,x_teste2,y_treino2,y_teste2 = train_test_split(d, dr,test_size= 0.3, random_state=0)

from sklearn.model_selection import train_test_split
x_treino3,x_teste3,y_treino3,y_teste3 = train_test_split(X_over,y_over,test_size=0.3,stratify=y_over,random_state=42)
antes = x_treino2.copy()
x_treino2['stroke'] = y_treino2

depois = x_treino3.copy()
x_treino3['stroke'] = y_treino3



antes = x_treino2.copy()
antes['stroke'] = y_treino2

antes_teste = x_teste2.copy()
antes_teste['stroke'] = y_teste2


depois = x_treino3.copy()
depois['stroke'] = y_treino3
depois_teste = x_teste3.copy()
depois_teste['stroke'] = y_teste3


def comparar(x,v):
    return len(x[x['stroke'] == v])

print("antes:",comparar(antes,1)," de ", comparar(antes,1)+comparar(antes,0))
print("Depois:",comparar(depois,1)," de ", comparar(depois,1)+comparar(x_treino3,0))

print('\n\n')

print("antes:",comparar(antes_teste,1)," de ", comparar(antes_teste,1)+comparar(antes_teste,0))
print("Depois:",comparar(depois_teste,1)," de ", comparar(depois_teste,1)+comparar(depois_teste,0))

"""

'\nx_treino2,x_teste2,y_treino2,y_teste2 = train_test_split(d, dr,test_size= 0.3, random_state=0)\n\nfrom sklearn.model_selection import train_test_split\nx_treino3,x_teste3,y_treino3,y_teste3 = train_test_split(X_over,y_over,test_size=0.3,stratify=y_over,random_state=42)\nantes = x_treino2.copy()\nx_treino2[\'stroke\'] = y_treino2\n\ndepois = x_treino3.copy()\nx_treino3[\'stroke\'] = y_treino3\n\n\n\nantes = x_treino2.copy()\nantes[\'stroke\'] = y_treino2\n\nantes_teste = x_teste2.copy()\nantes_teste[\'stroke\'] = y_teste2\n\n\ndepois = x_treino3.copy()\ndepois[\'stroke\'] = y_treino3\ndepois_teste = x_teste3.copy()\ndepois_teste[\'stroke\'] = y_teste3\n\n\ndef comparar(x,v):\n    return len(x[x[\'stroke\'] == v])\n\nprint("antes:",comparar(antes,1)," de ", comparar(antes,1)+comparar(antes,0))\nprint("Depois:",comparar(depois,1)," de ", comparar(depois,1)+comparar(x_treino3,0))\n\nprint(\'\n\n\')\n\nprint("antes:",comparar(antes_teste,1)," de ", comparar(antes_teste,1)+comparar(antes_

16/16 - 0s - loss: 0.1514 - accuracy: 0.7691 - recall: 0.6333 - 36ms/epoch - 2ms/step
[0.15137970447540283, 0.7690802216529846, 0.6333333253860474]


AttributeError: module 'matplotlib.pyplot' has no attribute 'line'

In [61]:
1

1